<a href="https://colab.research.google.com/github/3sanju/eco-sport/blob/main/relieff1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
import random
import time
import math
import numpy as np
import numpy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import KFold 
from sklearn import metrics
from sklearn.metrics import confusion_matrix, make_scorer, classification_report
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import precision_score, recall_score
import seaborn as sns

In [4]:
from google.colab import files
uploaded=files.upload();
data=pd.read_csv('/content/liver.csv')
data.drop('Patient Id',inplace=True,axis=1)
data.drop('index',inplace=True,axis=1)
data

Saving liver.csv to liver.csv


,Age,Gender,Air Pollution,Alcohol use,Dust Allergy,OccuPational Hazards,Genetic Risk,chronic Lung Disease,Balanced Diet,Obesity,...,Fatigue,Weight Loss,Shortness of Breath,Wheezing,Swal0ing Difficulty,Clubbing of Finger Nails,Frequent Cold,Dry Cough,Snoring,Level
0,33,1,2,4,5,4,3,2,2,4,...,3,4,2,2,3,1,2,3,4,0
1,17,1,3,1,5,3,4,2,2,2,...,1,3,7,8,6,2,1,7,2,1
2,35,1,4,5,6,5,5,4,6,7,...,8,7,9,2,1,4,6,7,2,1
3,37,1,7,7,7,7,6,7,7,7,...,4,2,3,1,4,5,6,7,5,1
4,46,1,6,8,7,7,7,6,7,7,...,3,2,4,1,4,2,4,2,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,44,1,6,7,7,7,7,6,7,7,...,5,3,2,7,8,2,4,5,3,1
996,37,2,6,8,7,7,7,6,7,7,...,9,6,5,7,2,4,3,1,4,1
997,25,2,4,5,6,5,5,4,6,7,...,8,7,9,2,1,4,6,7,2,1
998,18,2,6,8,7,7,7,6,7,7,...,3,2,4,1,4,2,4,2,3,1


In [5]:

def reliefF(df,n_neighbours=10,instances_to_select=10):
  import numpy as np
  feature=df.iloc[:,:-1]
  targetClass=df.iloc[:,-1]
  m,n=feature.shape

  weights=np.zeros(n,dtype='int')

  classes=np.unique(targetClass)

  m2=instances_to_select #number of features to pickup randomly from the 
  k=n_neighbours #number of neighbours to consider

  instances=np.array(list(range(1,m)))
  

  minimums=np.min(feature.values,axis=0)
  maximums=np.max(feature.values,axis=0)

  differ=np.subtract(maximums,minimums)


  for i in range(m2):
    chosen=np.random.choice(instances[:-1])
    instances=np.delete(instances,np.where(instances==chosen))

    rI=feature.iloc[chosen,:].values

    instanceClass=targetClass[chosen]
    probIClass=len(np.where(targetClass==instanceClass)[0])/m #getting the probaility of choosing this class

    hit=[]
    miss={}

    low,high,tem1,tem2=0,m,(chosen-1),(chosen+1)
    hitFlag=True
    missFlag=True
    
    
    while(hitFlag==True):

      if targetClass[tem1]==instanceClass:
        hit.append(tem1)
      if targetClass[tem2]==instanceClass:
        hit.append(tem2)
      if len(hit)==k:
        hitFlag=False
      if tem1>0:
        tem1-=1
      if tem2<m-1:
        tem2+=1
    for x in classes:
      if x==instanceClass:
        continue
      #print(instanceClass,x)
      cli=max(np.where(targetClass==x)[0]) #finding the last instance of the class x
      if cli>chosen:
        tem=min(np.where(targetClass==x)[0])
        miss[x]=list(range(tem,tem+k))
      else:
        tem=max(np.where(targetClass==x)[0])
        miss[x]=list(range(cli-k+1,cli))

    #print("Chosen-",chosen,"Hits-",hit,"Misses-",miss)

    totalHit=np.zeros(n,dtype='int')

    for hit in range(k):
      hI=feature.iloc[hit,:].values
      dRH=np.divide(np.abs(np.subtract(rI,hI)),differ)
      dRH=dRH/(m2*k)
      totalHit=np.add(totalHit,dRH)

    totalMiss=np.zeros(n,dtype='int')

    for eachClass in miss:

      tMiss=np.zeros(n,dtype='int')
      pclass=len(np.where(targetClass==eachClass)[0])/m #getting the probability of getting this class
      postProb=pclass/(1-probIClass) #calculating the posterior probanility of getting this class

      for eachMiss in miss[eachClass]:
        mI=feature.iloc[eachMiss,:].values
        dRM=np.divide(np.abs(np.subtract(rI,mI)),differ)
        dRM=dRM/(m2*k)
        tMiss=np.add(tMiss,dRM)

      totalMiss=np.add(totalMiss,(tMiss*postProb))

    weights=np.add(weights,totalMiss)
    weights=np.subtract(weights,totalHit) 
    
  return weights


In [8]:
data.head()

,Age,Gender,Air Pollution,Alcohol use,Dust Allergy,OccuPational Hazards,Genetic Risk,chronic Lung Disease,Balanced Diet,Obesity,...,Fatigue,Weight Loss,Shortness of Breath,Wheezing,Swal0ing Difficulty,Clubbing of Finger Nails,Frequent Cold,Dry Cough,Snoring,Level
0,33,1,2,4,5,4,3,2,2,4,...,3,4,2,2,3,1,2,3,4,0
1,17,1,3,1,5,3,4,2,2,2,...,1,3,7,8,6,2,1,7,2,1
2,35,1,4,5,6,5,5,4,6,7,...,8,7,9,2,1,4,6,7,2,1
3,37,1,7,7,7,7,6,7,7,7,...,4,2,3,1,4,5,6,7,5,1
4,46,1,6,8,7,7,7,6,7,7,...,3,2,4,1,4,2,4,2,3,1


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   Age                       1000 non-null   int64
 1   Gender                    1000 non-null   int64
 2   Air Pollution             1000 non-null   int64
 3   Alcohol use               1000 non-null   int64
 4   Dust Allergy              1000 non-null   int64
 5   OccuPational Hazards      1000 non-null   int64
 6   Genetic Risk              1000 non-null   int64
 7   chronic Lung Disease      1000 non-null   int64
 8   Balanced Diet             1000 non-null   int64
 9   Obesity                   1000 non-null   int64
 10  Smoking                   1000 non-null   int64
 11  Passive Smoker            1000 non-null   int64
 12  Chest Pain                1000 non-null   int64
 13  Coughing of Blood         1000 non-null   int64
 14  Fatigue                   1000 non-null  

In [11]:
#Examine the number of unique values for each variable/feature. Note that missing values are not being include as unique values.
unique_count=data.nunique()
unique_count

Age                         40
Gender                       2
Air Pollution                8
Alcohol use                  8
Dust Allergy                 8
OccuPational Hazards         8
Genetic Risk                 7
chronic Lung Disease         7
Balanced Diet                7
Obesity                      7
Smoking                      8
Passive Smoker               8
Chest Pain                   9
Coughing of Blood            9
Fatigue                      8
Weight Loss                  8
Shortness of Breath          8
Wheezing                     8
Swal0ing Difficulty          8
Clubbing of Finger Nails     9
Frequent Cold                7
Dry Cough                    7
Snoring                      7
Level                        2
dtype: int64

In [23]:
#Evaluate missingness and data availability
print("Missing Value Counts")
missing_count = unique_count.isnull().sum()
missing_count

Missing Value Counts


0

AttributeError: ignored

In [50]:
#Code to automatically select and generate the most appropriate exploratory data visualization pairing each indepentend variable (i.e. feature) with the dependent variable (i.e. outcome) 
def graph_test_selector(feature_name, outcome_name, data, p_val_dict):

    if data[feature_name].dtype == 'object' and data[outcome_name].dtype == 'object': #Feature and Outcome are discrete/categorical/binary
        #Generate contingency table count bar plot. ------------------------------------------------------------------------
        #Calculate Contingency Table - Counts
        print('#################')
        table = pd.crosstab(data[feature_name], data[outcome_name])
        print(table)
        geom_bar_data = pd.DataFrame(table)
        mygraph = geom_bar_data.plot(kind='bar')
        plt.ylabel('Count')
        new_feature_name = feature_name.replace(" ","") #Deal with the dataset specific characters causing problems in this dataset.
        new_feature_name = new_feature_name.replace("*","") #Deal with the dataset specific characters causing problems in this dataset.
        new_feature_name = new_feature_name.replace("/","") #Deal with the dataset specific characters causing problems in this dataset.
        plt.savefig('Exploratory_Plots/BarPlot_'+str(new_feature_name)+'_'+outcome_name+'.pdf')
        
        #Univariate association test (Chi Square Test of Independence - Non-parametric)
        c, p, dof, expected = scs.chi2_contingency(table)
        print("Chi Square P-value = "+str(p))
        
        #Identify and save features with 'significant' univariate association
        if p <= 0.05:
            p_val_dict[feature_name] = p
        
        
    elif data[feature_name].dtype == 'float64' and data[outcome_name].dtype == 'object': #Feature is continuous and Outcome is discrete/categorical/binary
        #Generate boxplot-----------------------------------------------------------------------------------------------------
        print('#################')
        print(feature_name)
        mygraph = data.boxplot(column=feature_name,by=outcome_name)
        plt.ylabel(feature_name)
        plt.title('')
        new_feature_name = feature_name.replace(" ","") #Deal with the dataset specific characters causing problems in this dataset.
        new_feature_name = new_feature_name.replace("*","") #Deal with the dataset specific characters causing problems in this dataset.
        new_feature_name = new_feature_name.replace("/","") #Deal with the dataset specific characters causing problems in this dataset.
        plt.savefig('Exploratory_Plots/BoxPlot_'+str(new_feature_name)+'_'+outcome_name+'.pdf')
        
        #Univariate association test (Mann-Whitney Test - Non-parametric)
        c, p = scs.mannwhitneyu(x=data[feature_name].loc[data[outcome_name] == 0],y=data[feature_name].loc[data[outcome_name] == 1])
        print("Mann-Whitney P-value = "+str(p))
        print("Mann-Whitney U-Statistic = "+str(c))

        #Identify and save features with 'significant' univariate association
        if p <= 0.05:
            p_val_dict[feature_name] = p
        
    elif data[feature_name].dtype == 'float64' and data[outcome_name].dtype == 'float64': #Feature is continuous and Outcome is discrete/categorical/binary
        #Generate scatterplot-------------------------------------------------------------------------------------------------
        print('#################')
        print(feature_name)
        mygraph = data.plot(x=feature_name,y=outcome_name,kind='scatter')
        new_feature_name = feature_name.replace(" ","") #Deal with the dataset specific characters causing problems in this dataset.
        new_feature_name = new_feature_name.replace("*","") #Deal with the dataset specific characters causing problems in this dataset.
        new_feature_name = new_feature_name.replace("/","") #Deal with the dataset specific characters causing problems in this dataset.
        plt.savefig('Exploratory_Plots/ScatterPlot_'+str(new_feature_name)+'_'+outcome_name+'.pdf')

        #Univariate association test (Spearman Rank-Order Correlation - Non-parametric)
        p = scs.spearmanr(x=data[feature_name],y=data[outcome_name])
        print("Spearman Correlation P-value = " + str(p))

        #Identify and save features with 'significant' univariate association
        if p <= 0.05:
            p_val_dict[feature_name] = p        

    elif data[feature_name].dtype == 'object' and data[outcome_name].dtype == 'float64': #Feature is continuous and Outcome is discrete/categorical/binary
        #Generate boxplot------------------------------------------------------------------------------------------------------
        print('#################')
        print(feature_name)
        mygraph = data.boxplot(column=outcome_name,by=feature_name)
        plt.ylabel(outcome_name)
        plt.title('')
        new_feature_name = feature_name.replace(" ","") #Deal with the dataset specific characters causing problems in this dataset.
        new_feature_name = new_feature_name.replace("*","") #Deal with the dataset specific characters causing problems in this dataset.
        new_feature_name = new_feature_name.replace("/","") #Deal with the dataset specific characters causing problems in this dataset.
        plt.savefig('Exploratory_Plots/BoxPlot_'+str(new_feature_name)+'_'+outcome_name+'.pdf')
        
        #Univariate association test (Mann-Whitney Test - Non-parametric)
        c, p = scs.mannwhitneyu(x=data[outcome_name].loc[data[feature_name] == '0'],y=data[outcome_name].loc[data[feature_name] == '1'])
        print("Mann-Whitney P-value = "+str(p))
        print("Mann-Whitney U-Statistic = "+str(c))

        #Identify and save features with 'significant' univariate association
        if p <= 0.05:
            p_val_dict[feature_name] = p
            
    else:
        print("Variable type combination not found! Check that all variables have been properly cast as either 'object' or 'float64'!")

In [56]:
header = list(data)

In [57]:
#For the purposes of downstream ML recast all features previously assigned to be 'object' to 'int'
data[header[0:23]] = data[header[0:23]].astype(dtype='int')
data[['Class Attribute']] = data[['Class Attribute']].astype(dtype='int')
data.dtypes
#Partition data into a training and testing set using convenient scikit-learn command.
train, test = train_test_split(data, test_size=0.2, stratify=td[outcome_name],random_state=randSeed) # 20% test set size, stratify option used to ensure class ratio is maintained in the partitions, random seed specified for reproducibility
train.shape # Confirm dimensions of training set
test.shape #Confirm dimensions of testing set

TypeError: ignored